# AIM: Visualize reported availability of volunteers on greographical map

Background:
- In crisis situation, organizations might require rapid and non-bureaucratic low-level support. This code aims at generating a geographical map with the available volunteers overlayed


Requirements:
- retrieve data via json API
    table of anonymized volunteers with postcode (PLZ, later GPS coord), weekday, type (time range)
- PLZ coordinates in .xml downloaded from http://data.geo.admin.ch/ch.swisstopo-vd.ortschaftenverzeichnis_plz/data.zip (generalizeable to other countries??)



# Initialization
(install and import packages)

In [ ]:
pip install matplotlib

plotly_express [https://github.com/plotly/plotly_express/]

In [ ]:
!pip3 install plotly_express

separate extension jupyterlab-plotly for jupyter-lab plotly [https://plotly.com/python/getting-started/#jupyterlab-support-python-35] 

In [ ]:
!pip3 install jupyterlab==1.2 "ipywidgets>=7.5"

geopandas

In [4]:
!pip3 install geopandas

     |████████████████████████████████| 931kB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 4.6MB/s eta 0:00:01    |██████████████████▉             | 8.7MB 3.9MB/s eta 0:00:02
  Using cached https://files.pythonhosted.org/packages/e9/da/824b92d9942f4e472702488857914bdd50f73021efea15b4cad9aca8ecef/click_plugins-1.1.1-py2.py3-none-any.whl


## list python modules with version

In [5]:
!pip3 freeze

absl-py==0.8.1
adabound==0.0.5
adanet==0.7.0
agate==1.6.1
agate-dbf==0.2.1
agate-excel==0.2.3
agate-sql==0.5.4
aiohttp==3.6.0
alabaster==0.7.12
alembic==1.1.0
alibi==0.2.3
altair==3.2.0
aniso8601==7.0.0
annoy==1.16.0
ansiwrap==0.8.4
apache-libcloud==2.6.0
appdirs==1.4.3
arrow==0.15.2
asn1crypto==1.2.0
astetik==1.9.9
astor==0.8.0
astroid==2.2.5
astropy==3.2.1
async-generator==1.10
async-timeout==3.0.1
atomicwrites==1.3.0
attrs==19.3.0
audioread==2.1.8
Augmentor==0.2.6
Automat==0.7.0
autopep8==1.4.4
Babel==2.7.0
backcall==0.1.0
bandit==1.6.2
bayesian-optimization==1.0.1
bcrypt==3.1.7
beautifulsoup4==4.8.1
black==19.3b0
blaze==0.10.1
bleach==3.1.0
blis==0.2.4
blosc==1.8.1
bokeh==1.3.4
boltons==19.1.0
boto==2.49.0
boto3==1.9.228
botocore==1.12.228
bottle==0.12.17
Bottleneck==1.2.1
bpemb==0.3.0
bqplot==0.11.8
branca==0.3.1
bs4==0.0.1
bz2file==0.98
cachetools==3.1.1
Cartopy==0.17.0
category-encoders==2.0.0
certifi==2019.11.28
certipy==0.1.3
cffi==1.13.2
chainer==6.3.0
chances==0.1.6
chardet=

# code

## read volunteer data via API .json

In [11]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
from collections import Counter
import numpy as np

url = "https://staging.medicalinformatics.ch/api/v1/helpers"

payload = "{\n\t\"hash\": \"4a9cmGzA\"\n}"
headers = {
  'Accept': 'application/json',
  'Content-Type': 'application/json'
}

indf = json.loads(requests.request("POST", url, headers=headers, data = payload).text.encode('utf8'))
#df = json_normalize(json.loads(response))
 
print(indf)

[{'postcode': 7754, 'latitude': 85.094616, 'longitude': 41.236094, 'capacities': [{'key': 'MONDAY_FORENOON', 'weekday': 'MONDAY', 'type': 'FORENOON'}, {'key': 'MONDAY_AFTERNOON', 'weekday': 'MONDAY', 'type': 'AFTERNOON'}, {'key': 'MONDAY_EVENING', 'weekday': 'MONDAY', 'type': 'EVENING'}, {'key': 'TUESDAY_FORENOON', 'weekday': 'TUESDAY', 'type': 'FORENOON'}, {'key': 'TUESDAY_NIGHT', 'weekday': 'TUESDAY', 'type': 'NIGHT'}, {'key': 'WEDNESDAY_FORENOON', 'weekday': 'WEDNESDAY', 'type': 'FORENOON'}, {'key': 'WEDNESDAY_AFTERNOON', 'weekday': 'WEDNESDAY', 'type': 'AFTERNOON'}, {'key': 'WEDNESDAY_EVENING', 'weekday': 'WEDNESDAY', 'type': 'EVENING'}, {'key': 'THURSDAY_FORENOON', 'weekday': 'THURSDAY', 'type': 'FORENOON'}, {'key': 'THURSDAY_AFTERNOON', 'weekday': 'THURSDAY', 'type': 'AFTERNOON'}, {'key': 'THURSDAY_EVENING', 'weekday': 'THURSDAY', 'type': 'EVENING'}, {'key': 'THURSDAY_NIGHT', 'weekday': 'THURSDAY', 'type': 'NIGHT'}, {'key': 'FRIDAY_FORENOON', 'weekday': 'FRIDAY', 'type': 'FORENOO

## post-processing  to get count table

In [102]:
# list of available time blocks
def list_of_blocks(ld):
    return list(list(json_normalize(d)['key']) for d in ld)

#def list_of_counts(ld):
#    return list(Counter(json_normalize(d)['key']) for d in ld)
#    return list(pd.Series(json_normalize(json_normalize(df)['capacities'][0])['key']) for d in ld)  
#   Create a mapping of the tuples formed after converting json strings of list to a python list

def occurDict(items):
    l = []
    for j in range(items.shape[0]):
        d = Counter([item for sublist in cou.values[j] for item in sublist])
        l.append(d)
    return l

#def list_of_dicts(ld):
#    return dict([(list(d.values())[1], list(d.values())[0]) for d in ld])

# eventually to extract from table and/or user selection via UI
a = ["MONDAY", "TUESDAY","WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]
b = ["FORENOON", "AFTERNOON", "EVENING", "NIGHT"]
blocks = ['_'.join([x,y]) for x in a for y in b]

# fill a dataframe
'''
If you plan to do things inside a big loop (say 10M‌ records or so), you are better off
using a mixture of these two; fill a dataframe with iloc until the size gets around 1000,
then append it to the original dataframe, and empty the temp dataframe. This would boost
your performance by around 10 times.

''' 
df = pd.DataFrame(columns=['PLZ', 'lat', 'lon', 'cou'], dtype=float)

df['PLZ'] = json_normalize(indf)['postcode'].values
df['lat'] = json_normalize(indf)['latitude'].values
df['lon'] = json_normalize(indf)['longitude'].values             

cou = json_normalize(indf)['capacities'].apply(list_of_blocks)
#print(cou.values)
dic = occurDict(cou.values)
#print(dic[0]['MONDAY_FORENOON'])
# sum over the selected time blocks
su = []
for cl in dic:
    s = []
    for b in blocks:
        s.append(cl[b])
    su.append(sum(s))
    
#print(len(dic))
df['cou'] = su

# data frame from list of dictionaries 
#data = [{'a': 1, 'b': 2},{'a': 5, 'b': 10, 'c': 20}]
#df = pd.DataFrame(data)

print(df)


     PLZ        lat         lon  cou
0   7754  85.094616   41.236094   22
1   1302  46.577000    6.539000    4
2   2068  47.014000    6.972000   13
3   8154  47.477000    8.521000    6
4   7849  70.701631   20.585604   20
5   1018  46.539000    6.629000   19
6   1247  46.276000    6.223000   18
7   5030  11.879975   14.143830   12
8   9763 -12.673244  100.532762   27
9   8480 -54.534339  -35.443489   24
10  7156  46.809000    9.117000   14
11  2005  52.291561  155.434605    7
12  5972 -14.542960  -84.359244   13
13  2526 -87.714189 -149.282318   24
14  1393  79.983104   95.050325    3
15   881  25.366018 -119.658361   26
16  3815  46.634000    7.912000    3
17  2388  89.550834  135.142923   18
18  2383  78.961034   44.341181   17
19  3810  70.048142   94.436112   10
20  2945  18.850525  -76.632786    9
21  1061  46.647000    6.729000   10
22  7659  13.313448    0.741980   12
23  8063   0.000000    0.000000    2
24   776 -43.100161   -1.584395   18
25  3196 -82.952168  173.939064   20
2

In [83]:
print([item for sublist in cou.values[27] for item in sublist])

['TUESDAY_FORENOON', 'TUESDAY_AFTERNOON', 'SATURDAY_AFTERNOON']


In [10]:
print(indf)

[{'postcode': 2282, 'latitude': 57.254013, 'longitude': -13.846746, 'capacities': [{'key': 'MONDAY_NIGHT', 'weekday': 'MONDAY', 'type': 'NIGHT'}, {'key': 'TUESDAY_FORENOON', 'weekday': 'TUESDAY', 'type': 'FORENOON'}, {'key': 'WEDNESDAY_AFTERNOON', 'weekday': 'WEDNESDAY', 'type': 'AFTERNOON'}, {'key': 'WEDNESDAY_NIGHT', 'weekday': 'WEDNESDAY', 'type': 'NIGHT'}, {'key': 'FRIDAY_FORENOON', 'weekday': 'FRIDAY', 'type': 'FORENOON'}, {'key': 'SATURDAY_AFTERNOON', 'weekday': 'SATURDAY', 'type': 'AFTERNOON'}]}, {'postcode': 46, 'latitude': 67.857323, 'longitude': 94.643538, 'capacities': [{'key': 'MONDAY_AFTERNOON', 'weekday': 'MONDAY', 'type': 'AFTERNOON'}, {'key': 'MONDAY_EVENING', 'weekday': 'MONDAY', 'type': 'EVENING'}, {'key': 'MONDAY_NIGHT', 'weekday': 'MONDAY', 'type': 'NIGHT'}, {'key': 'TUESDAY_FORENOON', 'weekday': 'TUESDAY', 'type': 'FORENOON'}, {'key': 'TUESDAY_AFTERNOON', 'weekday': 'TUESDAY', 'type': 'AFTERNOON'}, {'key': 'TUESDAY_EVENING', 'weekday': 'TUESDAY', 'type': 'EVENING'}

## generate map

To plot on Mapbox maps with Plotly you may need a Mapbox account and a public Mapbox Access Token. See our Mapbox Map Layers documentation for more information. ??? [https://plotly.com/python/scattermapbox/]

fig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon",     color="peak_hour", size="car_hours",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)



In [103]:
import plotly.express as px

# template: https://plotly.com/python/mapbox-layers/
#us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")


#fig = px.scatter_mapbox(us_cities, lat="lat", lon="lon", hover_name="City", hover_data=["State", "Population"],
#                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig = px.scatter_mapbox(df, lat="lat", lon="lon", hover_name="PLZ",
#                        color_discrete_sequence=["fuchsia"], zoom=2, height=400,
#                        color="PLZ", 
                        color_discrete_sequence=["fuchsia"],
                        size="cou",
                        color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=2, height=400
                       )
                       
                        
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [ ]:
print(us_cities["lat"])

In [ ]:
print(df["latitude"])

In [1]:
!pip -V
!pip3 -V
!python -V

pip 19.3.1 from /opt/conda/lib/python3.6/site-packages/pip (python 3.6)
pip 19.3.1 from /opt/conda/lib/python3.6/site-packages/pip (python 3.6)
Python 3.6.9 :: Anaconda, Inc.


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    # make the map global rather than have it zoom in to
    # the extents of any plotted data

ax.set_extent((60, 150, 55, -25))

ax.stock_img()
ax.coastlines()

ax.tissot(facecolor='purple', alpha=0.8)

plt.show()